<a href="https://colab.research.google.com/github/jetsonmom/saterlite_project/blob/main/%EC%9D%B8%EC%B2%9C%EA%B3%B5%ED%95%AD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 필요한 라이브러리 설치 및 import
!pip install geopandas folium contextily
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import Point, Polygon
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정
plt.rcParams['font.family'] = 'DejaVu Sans'

In [ ]:
# 파일 업로드
from google.colab import files
uploaded = files.upload()

# 압축 해제
import zipfile
with zipfile.ZipFile('LSMD_CONT_LDREG_인천_중구.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

# Shapefile 읽기
gdf = gpd.read_file('LSMD_CONT_LDREG_28110_202508.shp', encoding='cp949')

print("데이터 크기:", gdf.shape)
print("좌표계:", gdf.crs)
print("컬럼:", gdf.columns.tolist())

In [ ]:
# WGS84로 좌표계 변환
gdf_wgs84 = gdf.to_crs('EPSG:4326')

# 면적 계산 (원본 투영좌표계에서)
gdf['area'] = gdf.geometry.area

print("좌표 범위:")
bounds = gdf_wgs84.total_bounds
print(f"경도: {bounds[0]:.6f} ~ {bounds[2]:.6f}")
print(f"위도: {bounds[1]:.6f} ~ {bounds[3]:.6f}")

In [ ]:
# 활주로 특성으로 찾기: 매우 긴 직사각형 형태
gdf['length'] = gdf.geometry.length
gdf['width_ratio'] = gdf['area'] / gdf['length']

runway_candidates = gdf[
    (gdf['area'] > 100000) &      # 10만㎡ 이상
    (gdf['length'] > 3000) &      # 둘레 3km 이상
    (gdf['width_ratio'] > 50)     # 어느 정도 폭이 있는
]

print(f"활주로 후보: {len(runway_candidates)}개")

In [ ]:
# 활주로 중심점을 먼저 확인해봅시다
if len(runway_candidates) > 0:
    # 가장 큰 활주로를 실제 활주로로 가정
    main_runway = runway_candidates.loc[runway_candidates['area'].idxmax()]
    runway_center_proj = main_runway.geometry.centroid

    # WGS84로 변환하여 좌표 확인
    runway_wgs84 = gpd.GeoSeries([runway_center_proj], crs=gdf.crs).to_crs('EPSG:4326')
    runway_lon = runway_wgs84.iloc[0].x
    runway_lat = runway_wgs84.iloc[0].y

    print(f"🛫 메인 활주로 중심: 경도 {runway_lon:.6f}, 위도 {runway_lat:.6f}")
    print(f"   면적: {main_runway['area']:.0f}㎡")
    print(f"   지번: {main_runway['JIBUN']}")

In [ ]:
# 계류장 기준을 대폭 완화
apron_candidates = gdf[
    (gdf['area'] > 20000) &  # 2만㎡ 이상으로 기준 대폭 낮춤
    (gdf.geometry.centroid.distance(runway_center_proj) < 3000)  # 3km 반경 (미터 단위)
]

print(f"항공기 계류장 후보 (기준 완화): {len(apron_candidates)}개")

if len(apron_candidates) > 0:
    # 면적 순으로 정렬
    apron_sorted = apron_candidates.sort_values('area', ascending=False)
    print("\n상위 10개 계류장 후보:")
    for idx, row in apron_sorted.head(10).iterrows():
        print(f"지번: {row['JIBUN']}, 면적: {row['area']:.0f}㎡")

In [ ]:
# 주요 분석 대상 좌표 정리
analysis_targets = {
    "메인_활주로": {
        "지번": "2840잡",
        "면적": 3249260,
        "중심": [126.441048, 37.480019],
        "설명": "메인 활주로 - 가장 큰 구역"
    },
    "계류장_1": {
        "지번": "2868-1잡",
        "면적": 3152707,
        "설명": "대형 계류장 후보 1"
    },
    "계류장_2": {
        "지번": "2851-19잡",
        "면적": 2964114,
        "설명": "대형 계류장 후보 2"
    }
}

# 각 대상의 정확한 좌표 계산
for name, info in analysis_targets.items():
    if name != "메인_활주로":  # 이미 계산된 것 제외
        # 해당 지번 찾기
        target_area = gdf[gdf['JIBUN'] == info['지번']]
        if len(target_area) > 0:
            # WGS84 좌표로 변환
            target_wgs84 = target_area.to_crs('EPSG:4326')
            bounds = target_wgs84.total_bounds
            center_lon = (bounds[0] + bounds[2]) / 2
            center_lat = (bounds[1] + bounds[3]) / 2

            info['중심'] = [center_lon, center_lat]
            info['영역'] = [bounds[0], bounds[1], bounds[2], bounds[3]]

            print(f"📍 {name} ({info['지번']})")
            print(f"   중심좌표: {center_lon:.6f}, {center_lat:.6f}")
            print(f"   면적: {info['면적']:,}㎡ ({info['면적']/10000:.1f}만㎡)")
            print(f"   Google Earth: https://earth.google.com/web/@{center_lat},{center_lon},500a,35y,0h,0t,0r")

In [ ]:
# Google Earth Engine 설치 및 설정
!pip install earthengine-api geemap

import ee
import geemap

# GEE 인증 (처음 실행시)
try:
    ee.Initialize()
    print("✅ Google Earth Engine 초기화 완료")
except:
    print("🔐 Google Earth Engine 인증 필요...")
    ee.Authenticate()
    ee.Initialize()
    print("✅ 인증 및 초기화 완료")

# 메인 활주로 영역 설정 (중심점 기준 2

In [ ]:
# 지적도 데이터만으로 인천공항 분석하기
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import Point

# 데이터 로드 (이미 했다면 생략)
gdf = gpd.read_file('LSMD_CONT_LDREG_28110_202508.shp', encoding='cp949')
gdf_wgs84 = gdf.to_crs('EPSG:4326')
gdf['area'] = gdf.geometry.area

# 인천공항 주요 시설 분석
print("🛫 인천공항 지적도 기반 시설 분석")
print("=" * 50)

# 1. 활주로/택시웨이 (매우 큰 잡종지)
runways = gdf[
    (gdf['JIBUN'].str.contains('잡', na=False)) &
    (gdf['area'] > 1000000)  # 100만㎡ 이상
].sort_values('area', ascending=False)

print(f"\n🛬 활주로/택시웨이 ({len(runways)}개):")
for idx, row in runways.head(5).iterrows():
    # WGS84 좌표 계산
    bounds = gdf_wgs84.loc[idx].geometry.bounds
    center_lon = (bounds[0] + bounds[2]) / 2
    center_lat = (bounds[1] + bounds[3]) / 2

    print(f"   {row['JIBUN']}: {row['area']:,.0f}㎡")
    print(f"   중심좌표: {center_lon:.6f}, {center_lat:.6f}")

    # 예상 항공기 수용량 (대형 공항 기준)
    if '2840' in row['JIBUN']:
        print(f"   → 메인 활주로 추정")
        main_runway_center = [center_lon, center_lat]
    elif '2868' in row['JIBUN']:
        print(f"   → 터미널 계류장 추정 (약 {row['area']//5000:.0f}대 수용가능)")

# 2. 터미널 건물 (대지)
terminals = gdf[
    (gdf['JIBUN'].str.contains('대', na=False)) &
    (gdf['area'] > 10000)  # 1만㎡ 이상
].sort_values('area', ascending=False)

print(f"\n🏢 터미널 건물 ({len(terminals)}개):")
for idx, row in terminals.head(3).iterrows():
    bounds = gdf_wgs84.loc[idx].geometry.bounds
    center_lon = (bounds[0] + bounds[2]) / 2
    center_lat = (bounds[1] + bounds[3]) / 2

    print(f"   {row['JIBUN']}: {row['area']:,.0f}㎡")
    print(f"   중심좌표: {center_lon:.6f}, {center_lat:.6f}")

# 3. 주차장 (잡종지 중 중간 크기)
parking_areas = gdf[
    (gdf['JIBUN'].str.contains('잡', na=False)) &
    (gdf['area'] > 50000) & (gdf['area'] < 500000)  # 5-50만㎡
].sort_values('area', ascending=False)

print(f"\n🚗 주차장 후보 ({len(parking_areas)}개):")
for idx, row in parking_areas.head(5).iterrows():
    bounds = gdf_wgs84.loc[idx].geometry.bounds
    center_lon = (bounds[0] + bounds[2]) / 2
    center_lat = (bounds[1] + bounds[3]) / 2

    estimated_cars = row['area'] // 25  # 차량 1대당 25㎡
    print(f"   {row['JIBUN']}: {row['area']:,.0f}㎡ (약 {estimated_cars:,.0f}대)")
    print(f"   중심좌표: {center_lon:.6f}, {center_lat:.6f}")

In [ ]:
# 인천공항 주요 시설 종합 분석 및 시각화
fig, ax = plt.subplots(figsize=(25, 20))

# 전체 지적도 (연한 파란색)
gdf_wgs84.plot(ax=ax, alpha=0.3, color='lightblue', edgecolor='gray', linewidth=0.1)

# 1. 메인 활주로 (빨간색)
main_runway = gdf[gdf['JIBUN'] == '2840잡']
if len(main_runway) > 0:
    main_runway_wgs84 = main_runway.to_crs('EPSG:4326')
    main_runway_wgs84.plot(ax=ax, color='red', alpha=0.8, edgecolor='black', linewidth=3)

# 2. 터미널 계류장 (주황색)
apron1 = gdf[gdf['JIBUN'] == '2868-1잡']
if len(apron1) > 0:
    apron1_wgs84 = apron1.to_crs('EPSG:4326')
    apron1_wgs84.plot(ax=ax, color='orange', alpha=0.8, edgecolor='black', linewidth=3)

# 3. 기타 계류장들 (노란색)
other_aprons = gdf[gdf['JIBUN'].isin(['2851-19잡', '2851-31잡', '2869잡'])]
if len(other_aprons) > 0:
    other_aprons_wgs84 = other_aprons.to_crs('EPSG:4326')
    other_aprons_wgs84.plot(ax=ax, color='yellow', alpha=0.7, edgecolor='black', linewidth=2)

# 4. 주차장 상위 5개 (초록색)
top_parking_jibuns = ['2955-26 잡', '2955-8 잡', '82-1잡', '2840-10잡', '2837잡']
top_parking = gdf[gdf['JIBUN'].isin(top_parking_jibuns)]
if len(top_parking) > 0:
    top_parking_wgs84 = top_parking.to_crs('EPSG:4326')
    top_parking_wgs84.plot(ax=ax, color='green', alpha=0.7, edgecolor='black', linewidth=2)

# 범례 및 제목
plt.title('인천공항 지적도 기반 시설 분석\n빨강:활주로, 주황:터미널계류장, 노랑:기타계류장, 초록:주차장', fontsize=18)
plt.xlabel('경도', fontsize=14)
plt.ylabel('위도', fontsize=14)

# 범례 추가
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='red', label='메인 활주로'),
    Patch(facecolor='orange', label='터미널 계류장'),
    Patch(facecolor='yellow', label='기타 계류장'),
    Patch(facecolor='green', label='주차장')
]
ax.legend(handles=legend_elements, loc='upper right', fontsize=12)

plt.grid(True, alpha=0.3)
plt.show()

# 분석 결과 요약
print("\n📊 인천공항 분석 결과 요약")
print("=" * 60)
print(f"🛫 메인 활주로: 2840잡 (325만㎡)")
print(f"✈️  터미널 계류장: 2868-1잡 (315만㎡, 약 630대 항공기)")
print(f"🚗 최대 주차장: 2955-26 잡 (38만㎡, 약 15,393대)")
print(f"🏢 최대 터미널: 2955-74 대 (46만㎡)")

print(f"\n📍 주요 좌표:")
print(f"   활주로 중심: 126.441687, 37.480338")
print(f"   터미널 계류장: 126.417886, 37.468501")
print(f"   최대 주차장: 126.386706, 37.458719")

In [ ]:
# Planet Labs나 Sentinel 허브 사용
print("🛰️ 무료 위성영상 소스:")
print("1. Sentinel Hub: https://apps.sentinel-hub.com/")
print("2. Planet Labs (교육용): https://www.planet.com/")
print("3. Google Earth: https://earth.google.com/")

# 주요 분석 대상 좌표 정리
targets = {
    "터미널_계류장": "126.417886, 37.468501",
    "활주로": "126.441687, 37.480338",
    "주차장": "126.386706, 37.458719"
}

for name, coords in targets.items():
    print(f"\n📍 {name}: {coords}")
    print(f"   Google Earth: https://earth.google.com/web/@{coords.split(', ')[1]},{coords.split(', ')[0]},300a,35y,0h,0t,0r")

In [ ]:
from google.colab import files
import json
import geopandas as gpd

# AOI 파일 업로드
uploaded = files.upload()

In [ ]:
# GeoJSON 파일 읽기
with open('explorer-aoi.geojson', 'r', encoding='utf-8') as f:
    aoi_data = json.load(f)

print("✅ AOI 파일 로드 완료")
print(f"영역 타입: {aoi_data['type']}")
print(f"좌표계: {aoi_data.get('crs', 'WGS84')}")

# GeoPandas로 변환
aoi_gdf = gpd.read_file('explorer-aoi.geojson')
print(f"영역 개수: {len(aoi_gdf)}개")
print(f"총 면적: {aoi_gdf.geometry.area.sum():.6f} 제곱도")

In [ ]:
# AOI 상세 정보 (수정된 코드)
bounds = aoi_gdf.total_bounds
center_lon = (bounds[0] + bounds[2]) / 2
center_lat = (bounds[1] + bounds[3]) / 2

print(f"\n📍 Planet Labs AOI 분석 영역:")
print(f"경도 범위: {bounds[0]:.6f} ~ {bounds[2]:.6f}")
print(f"위도 범위: {bounds[1]:.6f} ~ {bounds[3]:.6f}")
print(f"중심점: {center_lon:.6f}, {center_lat:.6f}")
print(f"영역 크기: 약 {(bounds[2]-bounds[0])*111:.1f}km × {(bounds[3]-bounds[1])*111:.1f}km")
print(f"총 면적: {aoi_gdf.geometry.area.sum()*12100:.1f} km²")

# Planet Labs에서 위성영상 검색 준비
print(f"\n🛰️ 다음 단계:")
print(f"1. Planet Labs에서 이 AOI로 위성영상 검색")
print(f"2. 최근 구름 없는 고해상도 영상 다운로드")
print(f"3. Colab에서 항공기/차량 탐지 분석")

# 분석 대상 확인
print(f"\n✅ AOI 영역에 포함된 주요 시설:")
print(f"   - 메인 활주로 (보라색): 포함됨")
print(f"   - 터미널 계류장 (주황색): 포함됨")
print(f"   - 영역 크기: 적절함 (인천공항 핵심 구역)")

In [ ]:
# AOI 상세 정보 (수정된 코드)
bounds = aoi_gdf.total_bounds
center_lon = (bounds[0] + bounds[2]) / 2
center_lat = (bounds[1] + bounds[3]) / 2

print(f"\n📍 Planet Labs AOI 분석 영역:")
print(f"경도 범위: {bounds[0]:.6f} ~ {bounds[2]:.6f}")
print(f"위도 범위: {bounds[1]:.6f} ~ {bounds[3]:.6f}")
print(f"중심점: {center_lon:.6f}, {center_lat:.6f}")
print(f"영역 크기: 약 {(bounds[2]-bounds[0])*111:.1f}km × {(bounds[3]-bounds[1])*111:.1f}km")
print(f"총 면적: {aoi_gdf.geometry.area.sum()*12100:.1f} km²")

# Planet Labs에서 위성영상 검색 준비
print(f"\n🛰️ 다음 단계:")
print(f"1. Planet Labs에서 이 AOI로 위성영상 검색")
print(f"2. 최근 구름 없는 고해상도 영상 다운로드")
print(f"3. Colab에서 항공기/차량 탐지 분석")

# 분석 대상 확인
print(f"\n✅ AOI 영역에 포함된 주요 시설:")
print(f"   - 메인 활주로 (보라색): 포함됨")
print(f"   - 터미널 계류장 (주황색): 포함됨")
print(f"   - 영역 크기: 적절함 (인천공항 핵심 구역)")

In [ ]:
# 지적도 데이터 기반 최종 분석 결과
print("🛫 인천공항 지적도 기반 완전 분석")
print("=" * 60)

# 이미 파악한 주요 정보
airport_analysis = {
    "활주로": {
        "지번": "2840잡",
        "면적": "325만㎡",
        "좌표": "126.441687, 37.480338",
        "상태": "메인 활주로 - 대형 항공기 이착륙 가능"
    },
    "터미널_계류장": {
        "지번": "2868-1잡",
        "면적": "315만㎡",
        "좌표": "126.417886, 37.468501",
        "예상_항공기": "약 630대 수용 가능"
    },
    "주차장_최대": {
        "지번": "2955-26 잡",
        "면적": "38만㎡",
        "좌표": "126.386706, 37.458719",
        "예상_차량": "약 15,393대 수용 가능"
    },
    "터미널_건물": {
        "지번": "2955-74 대",
        "면적": "46만㎡",
        "좌표": "126.388190, 37.465195",
        "용도": "여객 터미널"
    }
}

for facility, info in airport_analysis.items():
    print(f"\n📍 {facility}:")
    for key, value in info.items():
        print(f"   {key}: {value}")

# 종합 결론
print(f"\n🎯 최종 분석 결과:")
print(f"✅ 인천공항 주요 시설 위치 파악 완료")
print(f"✅ 활주로 1개 (325만㎡) - 메인 활주로")
print(f"✅ 계류장 1개 (315만㎡) - 약 630대 항공기 주차 가능")
print(f"✅ 주차장 92개 - 최대 15,393대 차량 수용")
print(f"✅ 터미널 192개 - 최대 46만㎡ 규모")

# Google Earth 확인 링크
print(f"\n🔗 Google Earth로 실제 확인:")
print(f"   활주로: https://earth.google.com/web/@37.480338,126.441687,500a,35y,0h,0t,0r")
print(f"   계류장: https://earth.google.com/web/@37.468501,126.417886,500a,35y,0h,0t,0r")
print(f"   주차장: https://earth.google.com/web/@37.458719,126.386706,500a,35y,0h,0t,0r")

print(f"\n🎉 분석 완료! 추가 위성영상 다운로드 없이도 충분한 정보를 얻었습니다!")

In [ ]:
# 실제 비행기 카운팅용 좌표
print("🔍 비행기 세기 - 직접 확인 좌표")
print("=" * 40)

counting_areas = [
    {
        "구역": "터미널1 게이트",
        "좌표": "126.417886, 37.468501",
        "링크": "https://earth.google.com/web/@37.468501,126.417886,200a,0y,0t,0r"
    },
    {
        "구역": "터미널2 게이트",
        "좌표": "126.441687, 37.480338",
        "링크": "https://earth.google.com/web/@37.480338,126.441687,200a,0y,0t,0r"
    },
    {
        "구역": "화물 계류장",
        "좌표": "126.458882, 37.458983",
        "링크": "https://earth.google.com/web/@37.458983,126.458882,200a,0y,0t,0r"
    }
]

for area in counting_areas:
    print(f"\n📍 {area['구역']}")
    print(f"   좌표: {area['좌표']}")
    print(f"   링크: {area['링크']}")
    print(f"   → 이 링크 클릭해서 흰색 비행기들 직접 세기")

print(f"\n🎯 카운팅 방법:")
print(f"1. 각 링크를 새 탭에서 열기")
print(f"2. 흰색/은색 비행기 모양 찾기")
print(f"3. 수동으로 개수 세기")
print(f"4. 총합 계산")

In [ ]:
# Google Earth 이미지에서 비행기 자동 카운팅 시스템
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from google.colab import files
import io

class AircraftCounter:
    def __init__(self):
        self.aircraft_count = 0
        self.detected_aircraft = []

    def upload_and_analyze_image(self):
        """
        Google Earth 스크린샷 업로드 및 분석
        """
        print("🛫 비행기 자동 카운팅 시스템")
        print("=" * 40)
        print("1. Google Earth에서 인천공항 스크린샷 촬영")
        print("2. 아래에서 이미지 파일 업로드")
        print("3. 자동으로 비행기 개수 카운팅")

        # 파일 업로드
        uploaded = files.upload()

        for filename in uploaded.keys():
            print(f"\n📁 분석 파일: {filename}")
            result = self.detect_aircraft(filename)
            return result

    def detect_aircraft(self, image_path):
        """
        이미지에서 비행기 탐지 및 카운팅
        """
        try:
            # 이미지 로드
            img = cv2.imread(image_path)
            if img is None:
                print("❌ 이미지를 읽을 수 없습니다.")
                return None

            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            original_img = img_rgb.copy()

            print(f"✅ 이미지 로드 완료: {img.shape[1]} x {img.shape[0]} 픽셀")

            # 비행기 탐지 파이프라인
            aircraft_positions = self._find_aircraft(img)

            # 결과 시각화
            result_img = self._draw_detections(img_rgb, aircraft_positions)

            # 결과 출력
            self._display_results(original_img, result_img, aircraft_positions)

            return {
                'count': len(aircraft_positions),
                'positions': aircraft_positions,
                'original_image': original_img,
                'result_image': result_img
            }

        except Exception as e:
            print(f"❌ 오류 발생: {str(e)}")
            return None

    def _find_aircraft(self, img):
        """
        비행기 탐지 알고리즘
        """
        # 1. 그레이스케일 변환
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # 2. 가우시안 블러 (노이즈 제거)
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)

        # 3. 밝은 객체 탐지 (비행기는 보통 밝은 색상)
        # 적응적 임계값
        thresh = cv2.adaptiveThreshold(blurred, 255,
                                     cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                     cv2.THRESH_BINARY, 15, -3)

        # 4. 모폴로지 연산으로 노이즈 제거
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
        thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
        thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)

        # 5. 컨투어 찾기
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # 6. 비행기 후보 필터링
        aircraft_candidates = []

        for contour in contours:
            # 면적 필터 (비행기 크기)
            area = cv2.contourArea(contour)
            if area < 200 or area > 20000:  # 너무 작거나 큰 객체 제외
                continue

            # 경계 상자
            x, y, w, h = cv2.boundingRect(contour)

            # 종횡비 필터 (비행기는 길쭉함)
            aspect_ratio = w / h
            if aspect_ratio < 1.5 or aspect_ratio > 8.0:
                continue

            # 면적 대비 경계상자 비율 (비행기 모양)
            rect_area = w * h
            extent = area / rect_area
            if extent < 0.3:
                continue

            # 위치 정보 저장
            center_x = x + w // 2
            center_y = y + h // 2

            aircraft_candidates.append({
                'center': (center_x, center_y),
                'bbox': (x, y, w, h),
                'area': area,
                'aspect_ratio': aspect_ratio,
                'confidence': self._calculate_confidence(contour, area, aspect_ratio)
            })

        # 7. 신뢰도 기준으로 정렬 및 필터링
        aircraft_candidates.sort(key=lambda x: x['confidence'], reverse=True)

        # 8. NMS (Non-Maximum Suppression) - 중복 제거
        final_aircraft = self._apply_nms(aircraft_candidates)

        return final_aircraft

    def _calculate_confidence(self, contour, area, aspect_ratio):
        """
        비행기일 확률 계산
        """
        confidence = 0.5  # 기본값

        # 면적 점수 (적당한 크기)
        if 500 < area < 5000:
            confidence += 0.2

        # 종횡비 점수 (비행기 모양)
        if 2.0 < aspect_ratio < 6.0:
            confidence += 0.2

        # 컨투어 복잡도 (단순한 모양)
        perimeter = cv2.arcLength(contour, True)
        if perimeter > 0:
            circularity = 4 * np.pi * area / (perimeter * perimeter)
            if 0.1 < circularity < 0.8:
                confidence += 0.1

        return min(confidence, 1.0)

    def _apply_nms(self, candidates, overlap_threshold=0.3):
        """
        중복 탐지 제거 (Non-Maximum Suppression)
        """
        if not candidates:
            return []

        # 고신뢰도 후보만 선택
        high_conf_candidates = [c for c in candidates if c['confidence'] > 0.6]

        final_detections = []

        for candidate in high_conf_candidates:
            is_duplicate = False
            x1, y1, w1, h1 = candidate['bbox']

            for existing in final_detections:
                x2, y2, w2, h2 = existing['bbox']

                # 중복 영역 계산
                overlap_x = max(0, min(x1 + w1, x2 + w2) - max(x1, x2))
                overlap_y = max(0, min(y1 + h1, y2 + h2) - max(y1, y2))
                overlap_area = overlap_x * overlap_y

                area1 = w1 * h1
                area2 = w2 * h2
                union_area = area1 + area2 - overlap_area

                if union_area > 0:
                    iou = overlap_area / union_area
                    if iou > overlap_threshold:
                        is_duplicate = True
                        break

            if not is_duplicate:
                final_detections.append(candidate)

        return final_detections

    def _draw_detections(self, img, aircraft_list):
        """
        탐지 결과 시각화
        """
        result_img = img.copy()

        for i, aircraft in enumerate(aircraft_list):
            x, y, w, h = aircraft['bbox']
            center = aircraft['center']
            confidence = aircraft['confidence']

            # 경계상자 그리기
            cv2.rectangle(result_img, (x, y), (x + w, y + h), (255, 0, 0), 2)

            # 중심점 표시
            cv2.circle(result_img, center, 5, (0, 255, 0), -1)

            # 번호 및 신뢰도 표시
            label = f"{i+1}: {confidence:.2f}"
            cv2.putText(result_img, label, (x, y-10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)

        return result_img

    def _display_results(self, original, result, aircraft_list):
        """
        결과 출력 및 시각화
        """
        # 결과 통계
        print(f"\n🎯 비행기 탐지 결과:")
        print(f"   총 탐지된 비행기: {len(aircraft_list)}대")

        if aircraft_list:
            avg_confidence = np.mean([a['confidence'] for a in aircraft_list])
            print(f"   평균 신뢰도: {avg_confidence:.2f}")

            print(f"\n📊 탐지된 비행기 상세:")
            for i, aircraft in enumerate(aircraft_list):
                x, y = aircraft['center']
                conf = aircraft['confidence']
                area = aircraft['area']
                print(f"   {i+1}번: 위치({x}, {y}), 신뢰도 {conf:.2f}, 면적 {area}px")

        # 이미지 시각화
        fig, axes = plt.subplots(1, 2, figsize=(20, 10))

        axes[0].imshow(original)
        axes[0].set_title('원본 이미지', fontsize=14)
        axes[0].axis('off')

        axes[1].imshow(result)
        axes[1].set_title(f'비행기 탐지 결과 ({len(aircraft_list)}대)', fontsize=14)
        axes[1].axis('off')

        plt.tight_layout()
        plt.show()

        # 추가 분석
        if len(aircraft_list) > 0:
            self._analyze_distribution(aircraft_list, original.shape)

    def _analyze_distribution(self, aircraft_list, img_shape):
        """
        비행기 분포 분석
        """
        height, width = img_shape[:2]

        # 영역별 분포
        left_half = sum(1 for a in aircraft_list if a['center'][0] < width/2)
        right_half = len(aircraft_list) - left_half

        top_half = sum(1 for a in aircraft_list if a['center'][1] < height/2)
        bottom_half = len(aircraft_list) - top_half

        print(f"\n📍 비행기 분포 분석:")
        print(f"   좌측 영역: {left_half}대, 우측 영역: {right_half}대")
        print(f"   상단 영역: {top_half}대, 하단 영역: {bottom_half}대")

        # 크기별 분류
        sizes = [a['area'] for a in aircraft_list]
        if sizes:
            small = sum(1 for s in sizes if s < 1000)
            medium = sum(1 for s in sizes if 1000 <= s < 3000)
            large = sum(1 for s in sizes if s >= 3000)

            print(f"\n✈️ 크기별 분류:")
            print(f"   소형기: {small}대")
            print(f"   중형기: {medium}대")
            print(f"   대형기: {large}대")

# 사용법
def count_aircraft_in_google_earth():
    """
    Google Earth 이미지에서 비행기 카운팅 실행
    """
    counter = AircraftCounter()

    print("🛫 Google Earth 비행기 카운팅 시스템")
    print("=" * 50)
    print("사용법:")
    print("1. Google Earth에서 인천공항을 적당한 높이로 조정")
    print("2. 스크린샷 촬영 (Ctrl+Shift+S 또는 캡처)")
    print("3. 아래 업로드 버튼에서 이미지 선택")
    print("4. 자동으로 비행기 개수 카운팅!")
    print("\n💡 팁: 너무 높거나 낮지 않게, 비행기가 선명하게 보이는 고도로 조정")

    return counter.upload_and_analyze_image()

# 실행
if __name__ == "__main__":
    result = count_aircraft_in_google_earth()